In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.types import StructType, StructField, StringType
import os

In [2]:
#Spark Session using local(Standalone)
spark = SparkSession.builder \
     .master("local") \
     .appName("Pyspark SQL") \
     .config("spark.ui.port", "50032") \
     .getOrCreate()

In [32]:
df = spark.read.json("data/emp.json")

In [ ]:
emp_l = []

In [45]:
def convert_empdf_collection(df):
    l = []
    for row in df.collect():
        for i in range(0,len(row.empno)):
            l.append(
                (row.empno[i],
                 row.ename[i],
                 row.job[i],
                 row.mgr[i],
                 row.hiredate[i],
                 row.sal[i],
                 row.comm[i],
                 row.deptno[i])
            )
    return l

In [46]:
emp_t = convert_empdf_collection(df)
emp = spark.createDataFrame(emp_t,("empno","ename","job","mgr","hiredate","sal","comm","deptno"))
emp.show()

+-----+------+---------+------+----------+------+------+------+
|empno| ename|      job|   mgr|  hiredate|   sal|  comm|deptno|
+-----+------+---------+------+----------+------+------+------+
| 7369| SMITH|    CLERK|7902.0|1980-12-17| 800.0|  null|    20|
| 7499| ALLEN| SALESMAN|7698.0|1981-02-20|1600.0| 300.0|    30|
| 7876| ADAMS|    CLERK|7788.0|1983-01-12|1100.0|  null|    20|
| 7900| JAMES|    CLERK|7698.0|1981-12-03| 950.0|  null|    30|
| 7902|  FORD|  ANALYST|7566.0|1981-12-03|3000.0|  null|    20|
| 7934|MILLER|    CLERK|7782.0|1982-01-23|1300.0|  null|    10|
| 7521|  WARD| SALESMAN|7698.0|1981-02-22|1250.0| 500.0|    30|
| 7566| JONES|  MANAGER|7839.0|1981-04-02|2975.0|  null|    20|
| 7654|MARTIN| SALESMAN|7698.0|1981-09-28|1250.0|1400.0|    30|
| 7698| BLAKE|  MANAGER|7839.0|1981-05-01|2850.0|  null|    30|
| 7782| CLARK|  MANAGER|7839.0|1981-06-09|2450.0|  null|    10|
| 7788| SCOTT|  ANALYST|7566.0|1982-12-09|3000.0|  null|    20|
| 7839|  KING|PRESIDENT|  null|1981-11-1

In [52]:
emp[emp.deptno == 20].show()

+-----+-----+-------+------+----------+------+----+------+
|empno|ename|    job|   mgr|  hiredate|   sal|comm|deptno|
+-----+-----+-------+------+----------+------+----+------+
| 7369|SMITH|  CLERK|7902.0|1980-12-17| 800.0|null|    20|
| 7876|ADAMS|  CLERK|7788.0|1983-01-12|1100.0|null|    20|
| 7902| FORD|ANALYST|7566.0|1981-12-03|3000.0|null|    20|
| 7566|JONES|MANAGER|7839.0|1981-04-02|2975.0|null|    20|
| 7788|SCOTT|ANALYST|7566.0|1982-12-09|3000.0|null|    20|
+-----+-----+-------+------+----------+------+----+------+

